In [1]:
# import localcider
# from localcider.sequenceParameters import SequenceParameters
# import protfasta
# import glob
import pandas as pd
import numpy as np
import re
import scipy
from scipy import stats
import matplotlib.pyplot as plt
from collections import defaultdict 
import matplotlib.patches as mpatches
from matplotlib.patches import Patch

import ete3
from ete3 import TreeStyle, PhyloTree, Tree, faces, NodeStyle, AttrFace, TextFace
from ete3.treeview.faces import add_face_to_node

import seaborn as sns
lightblue , darkblue, lightgreen, darkgreen, pink =  '#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99'
# import editdistance

# import YeastAnalysisfunctions as YAF

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Helvetica"
plt.rcParams['pdf.fonttype'] = 42


In [4]:
def read_fasta(file):

    with open(file, "r") as f:
        lines = f.readlines()

    seq = ""
    seqs = []
    names = []

    for line in lines:
        if line.startswith(">"):
            names.append(line.strip().replace(">", ""))
            if len(names) > 1:
                seqs.append(seq.replace("*", ""))
                seq = ""
        else:
            seq += line.strip()

    seqs.append(seq.replace("*", ""))
    return pd.DataFrame({"name" : names, "aa_seq" : seqs})

# Looking at the full length sequences

In [37]:
FullOrthoDF =  pd.read_pickle('../data/FullOrthologDF_20240930')
FullOrthoDF

,SpeciesName,Seq,Length,WxxLF_loc,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess
0,Sordariomycetes_jgi|Acral2|2019554|gm1.4974_g,MALRIEVYNRIESSTASTALQRQDLRYTFRSNARAASGQANANYQA...,2928,1006,"[50849.75653537431, 50849.75653537431, 50849.7...","[0.0, 0.0, 0.2, 0.0, 0.0, 0.0, -0.2, 0.0, 0.2,...","[0.0, 0.0, 0.6666666666666666, 0.5466666666666...","[34049.34344014826, 34049.34344014826, 34049.3..."
1,Sordariomycetes_jgi|Acral2|2027520|fgenesh1_pg...,MWLVVRAGPSPLLQDLAARCHIDGMSMPLLHFDPPDFPLTGVALGI...,455,327,"[50311.68363475637, 50311.68363475637, 50311.6...","[0.0, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0, ...","[0.0, 0.0, 0.7933333333333333, 0.6511111111111...","[58366.833469168625, 58366.833469168625, 58366..."
2,Sordariomycetes_jgi|Acral2|2034848|fgenesh1_kg...,MSITELDDFTGFEGGASTAYSSPGAPAVFDLPGASNHVGTISPQDL...,222,94,"[64062.654182288825, 64062.654182288825, 75414...","[0.0, 0.0, -0.2, -0.2, -0.4, -0.6, -0.6, -0.4,...","[0.0, 0.0, 0.5311111111111111, 0.5733333333333...","[36846.26635132608, 36846.26635132608, 36846.2..."
3,Sordariomycetes_jgi|Acral2|2034849|fgenesh1_kg...,MSITGNYNQHFGAAGIISSHNYVLSELDDFTGFEGGASTAYSSPGA...,243,115,"[103421.45065118768, 103421.45065118768, 10342...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.6, 0.48, 0.4688888888888889, 0.29...","[98417.43715391349, 98417.43715391349, 98417.4..."
5,Sordariomycetes_jgi|Acral2|2047914|estExt_Gene...,MADTCGGSTPLKNFSQYGSQDRSLQQDRVVHGFHGSAAAGPSTFRS...,2943,1021,"[49761.33963354764, 49761.33963354764, 49761.3...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.5444444444444445, 0.4933333333333...","[36959.46539862907, 36959.46539862907, 36959.4..."
...,...,...,...,...,...,...,...,...
1207,Cimm_XP_012214147.1_CoccidioidesImmitisRS,MSTSNLPLDIGTLLDLSTDQFVEDLGSSSHSSLLDQDQLDQLINFN...,242,108,"[179044.502422884, 179044.502422884, 179044.50...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2, -0....","[0.0, 0.0, 0.4133333333333334, 0.4555555555555...","[215199.18805174268, 215199.18805174268, 21519..."
1208,Cpos_XP_003070205.1_CoccidioidesPosadasiiC735,MSTSNLPLGMVSLSASAVRLVANQRPDIGTLLDLSTDQYVEDLGSS...,260,126,"[56309.23879651313, 56309.23879651313, 56309.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.4133333333333334, 0.4555555555555...","[6138.0, 6138.0, 6138.0, 6138.0, 6138.0, 6138...."
1209,Mory_EHA48851.1_MagnaportheOryzae70_15,MNNTSDLGLDDFTAFGGGASAFPSPAMPGVFDIASTTASTMGTVSP...,239,101,"[165405.40017749005, 165405.40017749005, 16540...","[0.0, 0.0, 0.0, -0.2, -0.2, -0.2, -0.2, -0.4, ...","[0.0, 0.0, 0.3533333333333334, 0.2333333333333...","[156903.4915015713, 156903.4915015713, 156903...."
1210,Nfis_EAW24893.1_NeosartoryaFischeriNRRL181,MSTPNIAQDMPDFFGLPSNDFGDDFELSTEPTMLSPNQIPTGLMAV...,251,109,"[148604.20780984958, 148604.20780984958, 14860...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2, -0....","[0.0, 0.0, 0.39555555555555555, 0.453333333333...","[177960.75802414527, 177960.75802414527, 17796..."


In [39]:
FullOrthoDF["WxxLF"] = [bool(re.search("W..LF", seq)) for seq in FullOrthoDF["Seq"]]
FullOrthoDF["FF"] = [bool(re.search("FF", seq)) for seq in FullOrthoDF["Seq"]]
FullOrthoDF["FxF"] = [bool(re.search("F.F", seq)) for seq in FullOrthoDF["Seq"]]
FullOrthoDF["FxxF"] = [bool(re.search("F..F", seq)) for seq in FullOrthoDF["Seq"]]
FullOrthoDF["FxxxF"] = [bool(re.search("F...F", seq)) for seq in FullOrthoDF["Seq"]]
FullOrthoDF["MFxYxxL"] = [bool(re.search("M..Y..L", seq)) for seq in FullOrthoDF["Seq"]]

In [40]:
FullOrthoDF[["GCN4" in n for n in FullOrthoDF["SpeciesName"]]]

,SpeciesName,Seq,Length,WxxLF_loc,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess,WxxLF,FF,FxF,FxxF,FxxxF,MFxYxxL
1038,Saccharomycotina_sp|P03069|GCN4_YEAST,MSEYQPSLFALNPMGFSPLDGSKSTNENVSASTSTAKPMVGQLIFD...,281,119,"[46347.97933617003, 46347.97933617003, 46347.9...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.34, 0.26222222222222225, 0.262222...","[65239.94470515124, 65239.94470515124, 65239.9...",True,True,True,True,False,True


In [41]:
FullOrthoDF["FF"].value_counts()

FF
False    405
True      97
Name: count, dtype: int64

In [42]:
FullOrthoDF["FxF"].value_counts()

FxF
False    360
True     142
Name: count, dtype: int64

In [43]:
FullOrthoDF["FxxF"].value_counts()

FxxF
True     356
False    146
Name: count, dtype: int64

In [44]:
FullOrthoDF["MFxYxxL"].value_counts()

MFxYxxL
False    495
True       7
Name: count, dtype: int64

In [45]:
WLF_dict = {}

for i in FullOrthoDF.index:
    WLF_dict[FullOrthoDF.loc[i, "SpeciesName"].replace("#", "_")] = FullOrthoDF.loc[i, "WxxLF"]

In [46]:
FF_dict = {}

for i in FullOrthoDF.index:
    FF_dict[FullOrthoDF.loc[i, "SpeciesName"].replace("#", "_")] = FullOrthoDF.loc[i, "FF"]

In [47]:
FxF_dict = {}

for i in FullOrthoDF.index:
    FxF_dict[FullOrthoDF.loc[i, "SpeciesName"].replace("#", "_")] = FullOrthoDF.loc[i, "FxF"]

In [48]:
FxxF_dict = {}

for i in FullOrthoDF.index:
    FxxF_dict[FullOrthoDF.loc[i, "SpeciesName"].replace("#", "_")] = FullOrthoDF.loc[i, "FxxF"]

In [49]:
FxxxF_dict = {}

for i in FullOrthoDF.index:
    FxxxF_dict[FullOrthoDF.loc[i, "SpeciesName"].replace("#", "_")] = FullOrthoDF.loc[i, "FxxxF"]

In [50]:
MFxYxxL_dict = {}

for i in FullOrthoDF.index:
    MFxYxxL_dict[FullOrthoDF.loc[i, "SpeciesName"].replace("#", "_")] = FullOrthoDF.loc[i, "MFxYxxL"]

In [51]:
i = 0

motif_dict = FxxxF_dict
def mylayout(node):
    global i
    if node.is_leaf():
        name_face = AttrFace("name", fsize=60)
        node.add_face(name_face, column=0, position="aligned")

        nst = NodeStyle()
        nst["size"] = 0  # Size of the node circle, set to 0 to hide
        nst["vt_line_width"] = 10  # Vertical line thickness
        nst["hz_line_width"] = 10  # Horizontal line thickness
        nst["vt_line_color"] = "#333333"  # Vertical line color
        nst["hz_line_color"] = "#333333"  # Horizontal line color

        name = node.get_leaf_names()
        if name[0] == "Saccharomycotina_sp|P03069|GCN4_YEAST": 
            nst["bgcolor"] = "grey"

        if WLF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="red")
            faces.add_face_to_node(face, node, column=1, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=1, aligned=True)

        if FF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="orange")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        
        if FxF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="green")
            faces.add_face_to_node(face, node, column=3, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=3, aligned=True)

        if FxxF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="blue")
            faces.add_face_to_node(face, node, column=4, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=4, aligned=True)

        if MFxYxxL_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="purple")
            faces.add_face_to_node(face, node, column=5, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=5, aligned=True)
        
        node.set_style(nst)
                

t = PhyloTree("../data/phylogenetic_info/gcn4_gene_phylogeny.treefile", format=1)


# Removing things that aren't in the tree
filtered_names = [name for name in WLF_dict.keys() if t.search_nodes(name=name)]

t.prune(filtered_names)
ts = TreeStyle()

ts.layout_fn = mylayout
ts.mode = "c"
ts.show_leaf_name = False
# t.show(tree_style = ts)
t.render("../figures/motifs_on_gene_tree.pdf", tree_style = ts)

{'nodes': [[11754.612759122248,
   12569.169384904124,
   11758.693631206726,
   12573.250256988602,
   0,
   None],
  [11764.465245949348,
   12698.029981703135,
   11769.124816703112,
   12702.6895524569,
   2,
   None],
  [13433.686003687726,
   12202.81272582624,
   13438.878770690299,
   12208.00549282881,
   4,
   None],
  [13594.18624731749,
   11827.05035394347,
   13599.754222160682,
   11832.618328786664,
   5,
   None],
  [13468.943002492728,
   11331.471300234827,
   13474.543575578655,
   11337.071873320754,
   6,
   None],
  [12924.060128207626,
   10982.448253899503,
   12928.873800894173,
   10987.26192658605,
   7,
   None],
  [12104.097575562251,
   10952.282689139152,
   12109.028425529727,
   10957.213539106628,
   8,
   None],
  [11007.102370566308,
   11715.539852143134,
   11012.524451233456,
   11720.961932810284,
   9,
   None],
  [11086.880460202128,
   13735.564031920188,
   11092.505582891217,
   13741.189154609277,
   10,
   None],
  [13971.150210077281,
  

In [31]:
# Using this csv mapping because has tree names
name_key = pd.read_csv("../data/phylogenetic_info/top138_phyloinfo.csv")
name_key = name_key.rename({'SpeciesName' : 'Species'}, axis=1)
name_key = name_key.merge(FullOrthoDF, left_on='name', right_on='SpeciesName')

In [32]:
name_key[name_key["Species"] == 'Aspergillus nidulans']

,id,name,TreeLeaf,Validated,Species,NCBI Taxon,Note,AD_seq,full_sequence,SpeciesName,...,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess,WxxLF,FF,FxF,FxxF,FxxxF,MFxYxxL
472,4.0,Anid_AN3675,Aspnid1,Yes,Aspergillus nidulans,162425.0,NaN,PSDLFMDASAPPSTSFTDLSTPSFDSPGYFSQDTSPVFGADLDLAP...,MSTPNIPHQEFFDFTEGFGEDFTDPTMLSPHLVPTGIMASKDSLGD...,Anid_AN3675,...,"[38958.76569733663, 38958.76569733663, 38958.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2...","[0.0, 0.0, 0.39555555555555555, 0.453333333333...","[135260.90050398192, 135260.90050398192, 13526...",True,True,True,True,True,False
473,NaN,Anid_XP_661279.1_AspergillusNidulansFGSCA4,Aspnid1,Yes,Aspergillus nidulans,NaN,NaN,NaN,NaN,Anid_XP_661279.1_AspergillusNidulansFGSCA4,...,"[158091.99916887408, 158091.99916887408, 15809...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2,...","[0.0, 0.0, 0.39555555555555555, 0.453333333333...","[227146.83048752917, 227146.83048752917, 22714...",True,True,True,True,True,False


In [33]:
unique_species = name_key.sort_values('Length', ascending=False).drop_duplicates("SpeciesName", keep='first')
unique_species

,id,name,TreeLeaf,Validated,Species,NCBI Taxon,Note,AD_seq,full_sequence,SpeciesName,...,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess,WxxLF,FF,FxF,FxxF,FxxxF,MFxYxxL
115,NaN,Sordariomycetes_jgi|Acral2|2047914|estExt_Gene...,Acral2,Yes,Sodiomycetes alcalophilus,NaN,NaN,NaN,NaN,Sordariomycetes_jgi|Acral2|2047914|estExt_Gene...,...,"[49761.33963354764, 49761.33963354764, 49761.3...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.5444444444444445, 0.4933333333333...","[36959.46539862907, 36959.46539862907, 36959.4...",True,True,False,True,True,False
110,109.0,Sordariomycetes_jgi|Acral2|2019554|gm1.4974_g,Acral2,Yes,Sodiomycetes alcalophilus,398408.0,NaN,DLLIQDPYMSAPNSTALTALTSPSLYESPDFGYDVSPGFGSNDFDT...,MALRIEVYNRIESSTASTALQRQDLRYTFRSNARAASGQANANYQA...,Sordariomycetes_jgi|Acral2|2019554|gm1.4974_g,...,"[50849.75653537431, 50849.75653537431, 50849.7...","[0.0, 0.0, 0.2, 0.0, 0.0, 0.0, -0.2, 0.0, 0.2,...","[0.0, 0.0, 0.6666666666666666, 0.5466666666666...","[34049.34344014826, 34049.34344014826, 34049.3...",True,True,False,True,True,False
490,NaN,Pezizomycetes_jgi|Ascni1|352744|fgenesh1_pg.76...,Ascni1,Yes,Ascodesmis nigricans,NaN,NaN,NaN,NaN,Pezizomycetes_jgi|Ascni1|352744|fgenesh1_pg.76...,...,"[35776.33308908159, 35776.33308908159, 35776.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.6, 0.4, 0.4, ...","[0.0, 0.0, 0.3288888888888889, 0.2155555555555...","[31684.47651162896, 31684.47651162896, 31684.4...",True,True,True,True,True,False
55,136.0,Xylonomycetes_jgi|Trigu1|1079454|gm1.247_g,Trigu1,Yes,Trinosporium guianense,1196434.0,NaN,FRDPLASAPPSAAFTNLTSPSIFDSPDVAESFETSPLFANADHDLA...,MADALCGPSNPLQNLQKHTSVDRTLQQDRLVGPRHSPVQDFRSRSS...,Xylonomycetes_jgi|Trigu1|1079454|gm1.247_g,...,"[23437.5357660965, 23437.5357660965, 23437.535...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.6288888888888889, 0.6422222222222...","[29193.548646199746, 29193.548646199746, 29193...",True,True,False,True,True,False
15,137.0,Xylonomycetes_jgi|Xylhe1|264767|gm1.473_g,Xylhe1,Yes,Xylona heveae,1217826.0,NaN,LRDPLASAPPSAAFTNLTSPSIFDSPDVAESFETSPLFANADADLA...,MTDALCGPSNPLQHLQKHTSVDRTLQQDRLVGPRHSPVQDFRSQSQ...,Xylonomycetes_jgi|Xylhe1|264767|gm1.473_g,...,"[26567.768783458843, 26567.768783458843, 26567...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.5733333333333334, 0.5866666666666...","[32604.845533062748, 32604.845533062748, 32604...",True,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,21.0,Dothideomycetes_jgi|Didsa1|381788|estExt_Genew...,Didsa1,Yes,Didymocrea sadasivanii,372059.0,Splicing isoform / truncation (?) of other Did...,MFQGDSCFPTGTDTWYSLFPEEESRTVMPAVTTP,MFQGDSCFPTGTDTWYSLFPEEESRTVMPAVTTPLAAPALERTVSS...,Dothideomycetes_jgi|Didsa1|381788|estExt_Genew...,...,"[190922.41852774887, 161032.45391660908, 15386...","[0.0, 0.0, -0.2, -0.2, -0.2, -0.2, -0.2, 0.0, ...","[0.0, 0.0, 0.43999999999999995, 0.38, 0.373333...","[170312.56492479687, 170312.56492479687, 17031...",True,False,False,False,False,False
181,43.0,Eurotiomycetes_jgi|Penatra1|34162|e_gw1.2.1160.1,Penatra1,Yes,Penicillium atramentosum,36652.0,NaN,MFTDLDVAGHEDWPSLFDHSSEPLNAFDLATL,MFTDLDVAGHEDWPSLFDHSSEPLNAFDLATLDAAAAYSVEPKKPI...,Eurotiomycetes_jgi|Penatra1|34162|e_gw1.2.1160.1,...,"[261940.99846290916, 261910.30458108656, 26191...","[0.0, 0.0, -0.2, -0.4, -0.4, -0.4, -0.2, -0.2,...","[0.0, 0.0, 0.5955555555555556, 0.4755555555555...","[262143.0, 262143.0, 262143.0, 262143.0, 26214...",True,False,False,False,False,False
180,42.0,Eurotiomycetes_jgi|Penatra1|32388|gw1.2.1490.1,Penatra1,Yes,Penicillium atramentosum,36652.0,NaN,SPMFTDLDVAGHEDWPSLFDHSSEPLNAFDLATL,SPMFTDLDVAGHEDWPSLFDHSSEPLNAFDLATLDAAAAYSVEPKK...,Eurotiomycetes_jgi|Penatra1|32388|gw1.2.1490.1,...,"[173113.6358480084, 215442.77937959705, 230942...","[0.0, 0.0, 0.0, -0.2, -0.2, -0.4, -0.4, -0.4, ...","[0.0, 0.0, 0.5355555555555556, 0.4755555555555...","[238988.135652909, 238988.13565

In [34]:
WLF_dict = {}

for i in unique_species.index:
    WLF_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "WxxLF"]

In [35]:
FF_dict = {}

for i in unique_species.index:
    FF_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "FF"]

In [36]:
FxF_dict = {}

for i in unique_species.index:
    FxF_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "FxF"]

In [37]:
FxxF_dict = {}

for i in unique_species.index:
    FxxF_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "FxxF"]

In [38]:
name_dict = {}

for i in unique_species.index:
    name_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "Species"]

In [39]:
MFxYxxL_dict = {}

for i in unique_species.index:
    MFxYxxL_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "MFxYxxL"]

In [40]:
unique_species.columns

Index(['id', 'name', 'TreeLeaf', 'Validated', 'Species', 'NCBI Taxon', 'Note',
       'AD_seq', 'full_sequence', 'SpeciesName', 'Seq', 'Length', 'WxxLF_loc',
       'SmoothedActivites', 'LinearCharge', 'LinearHydrophobicityKD',
       'SmoothedActivitesLoess', 'WxxLF', 'FF', 'FxF', 'FxxF', 'FxxxF',
       'MFxYxxL'],
      dtype='object')

In [41]:
i = 0

def mylayout(node):
    global i
    if node.is_leaf():
        name_face = AttrFace("name", fsize=60)
        node.add_face(name_face, column=0, position="aligned")

        nst = NodeStyle()
        nst["size"] = 0  # Size of the node circle, set to 0 to hide
        nst["vt_line_width"] = 10  # Vertical line thickness
        nst["hz_line_width"] = 10  # Horizontal line thickness
        nst["vt_line_color"] = "#333333"  # Vertical line color
        nst["hz_line_color"] = "#333333"  # Horizontal line color

        name = node.get_leaf_names()
        if name[0] == "Saccharomycotina_sp|P03069|GCN4_YEAST": 
            nst["bgcolor"] = "black"
        if (name[0] in WLF_dict.keys()) or (name[0] in WLF_dict.keys()):
            if WLF_dict[name[0]] == 1:
                face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="red")
                faces.add_face_to_node(face, node, column=1, aligned=True)
            else: 
                face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
                faces.add_face_to_node(face, node, column=1, aligned=True)
        else:
            print(name)
        if (name[0] in FF_dict.keys()) or (name[0] in FF_dict.keys()):
            if FF_dict[name[0]] == 1:
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="orange")
                faces.add_face_to_node(face, node, column=2, aligned=True)
            else: 
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="white")
                faces.add_face_to_node(face, node, column=2, aligned=True)

        if (name[0] in FxF_dict.keys()) or (name[0] in FxF_dict.keys()):   
            if FxF_dict[name[0]] == 1:
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="green")
                faces.add_face_to_node(face, node, column=3, aligned=True)
            else: 
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="white")
                faces.add_face_to_node(face, node, column=3, aligned=True)

        if (name[0] in FxxF_dict.keys()) or (name[0] in FxxF_dict.keys()):   
            if FxxF_dict[name[0]] == 1:
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="blue")
                faces.add_face_to_node(face, node, column=4, aligned=True)
            else: 
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="white")
                faces.add_face_to_node(face, node, column=4, aligned=True)

        if (name[0] in MFxYxxL_dict.keys()) or (name[0] in MFxYxxL_dict.keys()):   
            if MFxYxxL_dict[name[0]] == 1:
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="purple")
                faces.add_face_to_node(face, node, column=5, aligned=True)
            else: 
                face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="white")
                faces.add_face_to_node(face, node, column=5, aligned=True)


        
        node.name =  name_dict[name[0]]
        node.set_style(nst)
                

# Note: This tree is missing some species
t = PhyloTree("../data/phylogenetic_info/myco-species-tree_added.nwk", format=1)

seq_names = ["'" + s + "'" for s in unique_species["Species"].values]

# Removing things that aren't in the tree
# filtered_names = [name for name in unique_species["Species"].values if t.search_nodes(name=name)] + [name for name in seq_names if t.search_nodes(name=name)]

# t.prune(filtered_names)
ts = TreeStyle()

ts.layout_fn = mylayout
ts.mode = "c"
ts.show_leaf_name = False
# t.show(tree_style = ts)
t.render("../figures/motifs_on_species_tree.pdf", tree_style = ts)

{'nodes': [[4321.339545693703,
   4147.654548304322,
   4326.951840570544,
   4153.266843181163,
   0,
   None],
  [4206.707074016735,
   4034.2145594312815,
   4211.368884918306,
   4038.8763703328536,
   1,
   None],
  [3884.77822779173,
   4189.741157524018,
   3889.3460222795784,
   4194.308952011866,
   2,
   None],
  [3861.896892918323,
   4381.154100860451,
   3866.999974729579,
   4386.257182671707,
   3,
   None],
  [3994.6121770399104,
   4757.592634102069,
   3999.9644862471,
   4762.944943309259,
   4,
   None],
  [4371.991978689417,
   4838.748511972646,
   4376.736620641208,
   4843.493153924438,
   5,
   None],
  [4554.4284857301745,
   4790.76924858274,
   4559.880466253369,
   4796.221229105935,
   6,
   None],
  [4634.644330414697,
   4773.756756450951,
   4640.23725626503,
   4779.349682301283,
   7,
   None],
  [4726.915836733148,
   4778.171444674273,
   4732.565480753284,
   4783.821088694408,
   8,
   None],
  [4817.158537704639,
   4777.636892050629,
   4822.811

# Looking at motifs in the central AD

In [5]:
cAD_seqs = read_fasta("../data/fasta_files/AllSeqs_IntegralAround_WxxLF_-50_+20_top138.fasta")

In [6]:
cAD_seqs

,name,aa_seq
0,Eurotiomycetes_jgi|Penatra1|34162|e_gw1.2.1160.1,MFTDLDVAGHEDWPSLFDHSSEPLNAFDLATL
1,Scas_Scas663.23,SEEQEDDQFMTIPPLNELDSNVVDAFFSSSTDSTPMFEFESLDESN...
2,Cten_EGV62856.1_CandidaTenuisATCC10573,ESAPGLSAPDSSLAFHSEVLDSVFSNDEAVDHTPMFDELDFMLESS...
3,Eurotiomycetes_jgi|Asptam1|203219|CE203218_32345,PKDLFMDASAPPSASFTDLSTPSFESPGYFSQDTSPMFATDMELGP...
4,Eurotiomycetes_jgi|Penbr2|60600|gm1.7763_g,ISPRDLMMDTSVPPSGTFTDLSTPSFESPGNFSQNASPMFTDMDLV...
...,...,...
133,Sordariomycetes_jgi|Hypfra2|223471|CE223470_11...,QDHFTSAPNSSAITNLTSPSMYGESPDLHDSYEVSPNYGGSDFDHG...
134,Blastocladiomycota_jgi|Catan2|1506241|gm1.11555_g,TSTSAPPSPVPSLELCAATSGRALSKSPSPFDFGAAPPADATLVVS...
135,Saccharomycotina_jgi|Ascru1|80718|fgenesh1_pm....,NSNRRNNAISHDDVVAPKQTISREELLLITDQVFSNDNVSIFDEKL...
136,Saccharomycotina_jgi|Hanpo2|10524|gm1.4782_g,DDHNADQALRSNPGLTISPFEIHSSVIGSIFEDPDAETAPMFEPRE...


In [33]:
cAD_seqs["WxxLF"] = [bool(re.search("W..LF", seq)) for seq in cAD_seqs["aa_seq"]]
cAD_seqs["FF"] = [bool(re.search("FF", seq)) for seq in cAD_seqs["aa_seq"]]
cAD_seqs["FxF"] = [bool(re.search("F.F", seq)) for seq in cAD_seqs["aa_seq"]]
cAD_seqs["FxxF"] = [bool(re.search("F..F", seq)) for seq in cAD_seqs["aa_seq"]]
cAD_seqs["FxxxF"] = [bool(re.search("F...F", seq)) for seq in cAD_seqs["aa_seq"]]
cAD_seqs["FxxxxF"] = [bool(re.search("F....F", seq)) for seq in cAD_seqs["aa_seq"]]
cAD_seqs["MFxYxxL"] = [bool(re.search("M..Y..L", seq)) for seq in cAD_seqs["aa_seq"]]

In [8]:
cAD_seqs[["Saccharomycotina_sp|P03069|GCN4_YEAST" in s for s in cAD_seqs['name']]]['aa_seq'].values[0]

'ALPQTATAPDAKTVLPIPELDDAVVESFFSSSTDSTPMFEYENLEDNSKEWTSLFDNDIPVTTDDVSLAD'

In [25]:
sum(cAD_seqs["FxxxxF"])

44

In [35]:
sum(cAD_seqs["MFxYxxL"])

5

In [27]:
sum(cAD_seqs["FxxF"])

36

In [34]:
WLF_dict = {}
FF_dict = {}
FxF_dict = {}
FxxF_dict = {}
FxxxF_dict = {}
FxxxxF_dict = {}
MFxYxxL_dict = {}

for i in cAD_seqs.index:
    name = cAD_seqs.loc[i, "name"]
    WLF_dict[name] = cAD_seqs.loc[i, "WxxLF"]
    FF_dict[name] = cAD_seqs.loc[i, "FF"]
    FxF_dict[name] = cAD_seqs.loc[i, "FxF"]
    FxxF_dict[name] = cAD_seqs.loc[i, "FxxF"]
    FxxxF_dict[name] = cAD_seqs.loc[i, "FxxxF"]
    FxxxxF_dict[name] = cAD_seqs.loc[i, "FxxxxF"]
    MFxYxxL_dict[name] = cAD_seqs.loc[i, "MFxYxxL"]


In [12]:
len(cAD_seqs["FxF"])

138

In [36]:
i = 0

def mylayout(node):
    global i
    if node.is_leaf():
        name_face = AttrFace("name", fsize=60)
        node.add_face(name_face, column=0, position="aligned")

        nst = NodeStyle()
        nst["size"] = 0  # Size of the node circle, set to 0 to hide
        # nst["vt_line_width"] = 10  # Vertical line thickness
        # nst["hz_line_width"] = 10  # Horizontal line thickness
        # nst["vt_line_color"] = "#333333"  # Vertical line color
        # nst["hz_line_color"] = "#333333"  # Horizontal line color

        name = node.get_leaf_names()
        if name[0] == "Saccharomycotina_sp|P03069|GCN4_YEAST": 
            nst["bgcolor"] = "grey"

        if WLF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="red")
            faces.add_face_to_node(face, node, column=1, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=1, aligned=True)

        if FF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="orange")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        
        if MFxYxxL_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="purple")
            faces.add_face_to_node(face, node, column=3, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=3, aligned=True)


        node.set_style(nst)
                

t = PhyloTree("../data/phylogenetic_info/gcn4_gene_phylogeny.treefile", format=1)

# Removing things that aren't in the tree
filtered_names = [name for name in cAD_seqs["name"].values if t.search_nodes(name=name)]

t.prune(filtered_names)
ts = TreeStyle()

ts.layout_fn = mylayout
ts.mode = "c"
ts.show_leaf_name = False
# t.show(tree_style = ts)
t.render("../figures/cAD_motifs_on_gene_tree.pdf", tree_style = ts)

{'nodes': [[5302.0434395800185,
   5329.845354066701,
   5306.4992747781,
   5334.301189264781,
   0,
   None],
  [5393.239601838741,
   5310.086264449836,
   5398.085414258086,
   5314.9320768691805,
   3,
   None],
  [5475.49898343946,
   5263.856647835576,
   5480.735346416303,
   5269.09301081242,
   4,
   None],
  [5520.138469609067,
   5190.7730112420895,
   5525.708466927832,
   5196.343008560854,
   5,
   None],
  [5485.415364714412,
   5116.591649334148,
   5491.01997241474,
   5122.196257034476,
   6,
   None],
  [5375.5567354803725,
   5051.972244289841,
   5380.194960476492,
   5056.610469285962,
   7,
   None],
  [5137.183372053259,
   5082.887291745167,
   5142.79615054776,
   5088.500070239668,
   8,
   None],
  [5022.044717657754,
   5491.141542421922,
   5027.466956120869,
   5496.563780885038,
   9,
   None],
  [5725.20445879183,
   5508.622587344962,
   5730.505047585993,
   5513.923176139126,
   10,
   None],
  [5757.072188824141,
   5497.896699596252,
   5762.26807

In [32]:
# For the F motifs only

i = 0

def mylayout(node):
    global i
    if node.is_leaf():
        name_face = AttrFace("name", fsize=60)
        node.add_face(name_face, column=0, position="aligned")

        nst = NodeStyle()
        nst["size"] = 0  # Size of the node circle, set to 0 to hide
        # nst["vt_line_width"] = 10  # Vertical line thickness
        # nst["hz_line_width"] = 10  # Horizontal line thickness
        # nst["vt_line_color"] = "#333333"  # Vertical line color
        # nst["hz_line_color"] = "#333333"  # Horizontal line color

        name = node.get_leaf_names()
        if name[0] == "Saccharomycotina_sp|P03069|GCN4_YEAST": 
            nst["bgcolor"] = "grey"

        if FF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="orange")
            faces.add_face_to_node(face, node, column=1, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=1, aligned=True)

        if FxF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="green")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        
        if FxxF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="blue")
            faces.add_face_to_node(face, node, column=3, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=3, aligned=True)
        
        if FxxxF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="pink")
            faces.add_face_to_node(face, node, column=4, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=4, aligned=True)

        if FxxxxF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="black")
            faces.add_face_to_node(face, node, column=5, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=5, aligned=True)


        node.set_style(nst)
                

t = PhyloTree("../data/phylogenetic_info/gcn4_gene_phylogeny.treefile", format=1)

# Removing things that aren't in the tree
filtered_names = [name for name in cAD_seqs["name"].values if t.search_nodes(name=name)]

t.prune(filtered_names)
ts = TreeStyle()

ts.layout_fn = mylayout
ts.mode = "c"
ts.show_leaf_name = False
# t.show(tree_style = ts)
t.render("../figures/cAD_FF_motifs_on_gene_tree.pdf", tree_style = ts)


{'nodes': [[5902.0434395800185,
   5929.845354066701,
   5906.4992747781,
   5934.301189264781,
   0,
   None],
  [5993.239601838741,
   5910.086264449836,
   5998.085414258086,
   5914.9320768691805,
   3,
   None],
  [6075.49898343946,
   5863.856647835576,
   6080.735346416303,
   5869.09301081242,
   4,
   None],
  [6120.138469609067,
   5790.7730112420895,
   6125.708466927832,
   5796.343008560854,
   5,
   None],
  [6085.415364714412,
   5716.591649334148,
   6091.01997241474,
   5722.196257034476,
   6,
   None],
  [5975.5567354803725,
   5651.972244289841,
   5980.194960476492,
   5656.610469285962,
   7,
   None],
  [5737.183372053259,
   5682.887291745167,
   5742.79615054776,
   5688.500070239668,
   8,
   None],
  [5622.044717657754,
   6091.141542421922,
   5627.466956120869,
   6096.563780885038,
   9,
   None],
  [6325.20445879183,
   6108.622587344962,
   6330.505047585993,
   6113.923176139126,
   10,
   None],
  [6357.072188824141,
   6097.896699596252,
   6362.26807

In [30]:
len(t.get_leaf_names())

131

In [9]:
name_key = pd.read_csv("../data/phylogenetic_info/top138_phyloinfo.csv")
name_key = name_key.rename({'SpeciesName' : 'Species'}, axis=1)
name_key = name_key.merge(cAD_seqs, on='name')
name_key["Length"] = name_key['aa_seq'].str.len()


In [18]:
unique_species = name_key.sort_values('Length', ascending=False).drop_duplicates("Species", keep='first')

name_dict = {}

for i in unique_species.index:
    name_dict[unique_species.loc[i, "TreeLeaf"]] = unique_species.loc[i, "Species"]

In [11]:
WLF_dict = {}
FF_dict = {}
FxF_dict = {}
FxxF_dict = {}
FxxxF_dict = {}
MFxYxxL_dict = {}

for i in unique_species.index:
    name = unique_species.loc[i, "TreeLeaf"]
    WLF_dict[name] = unique_species.loc[i, "WxxLF"]
    FF_dict[name] = unique_species.loc[i, "FF"]
    FxF_dict[name] = unique_species.loc[i, "FxF"]
    FxxF_dict[name] = unique_species.loc[i, "FxxF"]
    FxxxF_dict[name] = unique_species.loc[i, "FxxxF"]
    MFxYxxL_dict[name] = unique_species.loc[i, "MFxYxxL"]


In [13]:
i = 0

def mylayout(node):
    global i
    if node.is_leaf():
        name_face = AttrFace("name", fsize=60)
        node.add_face(name_face, column=0, position="aligned")

        nst = NodeStyle()
        nst["size"] = 0  # Size of the node circle, set to 0 to hide
        nst["vt_line_width"] = 10  # Vertical line thickness
        nst["hz_line_width"] = 10  # Horizontal line thickness
        nst["vt_line_color"] = "#333333"  # Vertical line color
        nst["hz_line_color"] = "#333333"  # Horizontal line color

        name = node.get_leaf_names()
        if name[0] == "Saccharomycotina_sp|P03069|GCN4_YEAST": 
            nst["bgcolor"] = "black"

        if WLF_dict[name[0]] == 1:
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="red")
            faces.add_face_to_node(face, node, column=1, aligned=True)
        else: 
            face = faces.RectFace(width=300, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=1, aligned=True)

        if FF_dict[name[0]] == 1:
            face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="orange")
            faces.add_face_to_node(face, node, column=2, aligned=True)
        else: 
            face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=2, aligned=True)
  
        if MFxYxxL_dict[name[0]] == 1:
            face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="purple")
            faces.add_face_to_node(face, node, column=3, aligned=True)
        else: 
            face = faces.RectFace(width=200, height=100, fgcolor="white", bgcolor="white")
            faces.add_face_to_node(face, node, column=3, aligned=True)

        node.set_style(nst)
                

# Note: This tree is missing some species
t = PhyloTree("../data/phylogenetic_info/myco-species-tree_added.nwk", format=1)

# Removing things that aren't in the tree - only looking at unique things in the cAD
filtered_names = [name for name in MFxYxxL_dict.keys() if t.search_nodes(name=name)] 
t.prune(filtered_names)

ts = TreeStyle()

ts.layout_fn = mylayout
ts.mode = "c"
ts.show_leaf_name = False
# t.show(tree_style = ts)
t.render("../figures/cAD_motifs_on_species_tree.pdf", tree_style = ts)

{'nodes': [[3550.7908441449167,
   3383.181744556296,
   3556.393824768542,
   3388.784725179921,
   0,
   None],
  [3436.349431548919,
   3275.2179576470144,
   3441.084823917381,
   3279.953350015477,
   1,
   None],
  [3128.244681088875,
   3435.565691602515,
   3132.707687907563,
   3440.0286984212034,
   2,
   None],
  [3111.7092917077703,
   3621.429507900459,
   3116.88603958458,
   3626.606255777269,
   3,
   None],
  [3253.3250689695387,
   3982.3958600319907,
   3258.6103871207124,
   3987.6811781831643,
   4,
   None],
  [3632.3435703148107,
   4045.1766147025505,
   3637.250211150917,
   4050.083255538657,
   5,
   None],
  [3809.2810407310376,
   3987.194388182234,
   3814.8148823250303,
   3992.7282297762267,
   6,
   None],
  [3883.7753076519757,
   3967.8160878842855,
   3889.407350779665,
   3973.4481310119754,
   7,
   None],
  [3966.7941150875704,
   3973.4970005271125,
   3972.4508378504775,
   3979.153723290019,
   8,
   None],
  [4045.68282813625,
   3978.88335941

# Put alignment on tree

In [19]:
from ete3 import TreeStyle
from ete3 import PhyloTree
from ete3 import Tree, faces, TreeStyle
from PyQt5       import QtCore
from PyQt5.QtWidgets import QGraphicsRectItem
from PyQt5.QtGui import QColor, QPen, QBrush

from PyQt5.QtWidgets import QGraphicsSimpleTextItem
from PyQt5.QtGui import QFont

from random import random

import pandas as pd

import os
import re

In [33]:
# Setting the background colors so that only the F is black
_aabgcolors = {
    'A':"#FFFFFF" ,
    'R':"#FFFFFF" ,
    'N':"#FFFFFF" ,
    'D':"#FFFFFF" ,
    'C':"#FFFFFF" ,
    'Q':"#FFFFFF" ,
    'E':"#FFFFFF" ,
    'G':"#FFFFFF" ,
    'H':"#FFFFFF" ,
    'I':"#FFFFFF" ,
    'L':"#FFFFFF" ,
    'K':"#FFFFFF" ,
    'M':"#FFFFFF" ,
    'F':"#000000" ,
    'P':"#FFFFFF" ,
    'S':"#FFFFFF" ,
    'T':"#FFFFFF" ,
    'W':"#FFFFFF" ,
    'Y':"#FFFFFF" ,
    'V':"#FFFFFF" ,
    'B':"#FFFFFF" ,
    'Z':"#FFFFFF" ,
    'X':"#FFFFFF",
    '.':"#FFFFFF",
    '-':"#FFFFFF",
    }

# Setting the foreground colors so that only F is white
_aafgcolors = {
    'A':"#000000" ,
    'R':"#000000" ,
    'N':"#000000" ,
    'D':"#000000" ,
    'C':"#000000" ,
    'Q':"#000000" ,
    'E':"#000000" ,
    'G':"#000000" ,
    'H':"#000000" ,
    'I':"#000000" ,
    'L':"#000000" ,
    'K':"#000000" ,
    'M':"#000000" ,
    'F':"#FFFFFF" ,
    'P':"#000000" ,
    'S':"#000000" ,
    'T':"#000000" ,
    'W':"#000000" ,
    'Y':"#000000" ,
    'V':"#000000" ,
    'B':"#000000" ,
    'Z':"#000000" ,
    'X':"#000000",
    '.':"#000000",
    '-':"#000000",
}

_aabgcolors_highlight = {
    'A':"#F3C4FF" ,
    'R':"#F3C4FF" ,
    'N':"#F3C4FF" ,
    'D':"#F3C4FF" ,
    'C':"#F3C4FF" ,
    'Q':"#F3C4FF" ,
    'E':"#F3C4FF" ,
    'G':"#F3C4FF" ,
    'H':"#F3C4FF" ,
    'I':"#F3C4FF" ,
    'L':"#F3C4FF" ,
    'K':"#F3C4FF" ,
    'M':"#F3C4FF" ,
    'F':"#000000" ,
    'P':"#F3C4FF" ,
    'S':"#F3C4FF" ,
    'T':"#F3C4FF" ,
    'W':"#F3C4FF" ,
    'Y':"#F3C4FF" ,
    'V':"#F3C4FF" ,
    'B':"#F3C4FF" ,
    'Z':"#F3C4FF" ,
    'X':"#F3C4FF",
    '.':"#F3C4FF",
    '-':"#F3C4FF",
    }

_aabgcolors_highlight_sc = {
    'A':"#C4EFFF" ,
    'R':"#C4EFFF" ,
    'N':"#C4EFFF" ,
    'D':"#C4EFFF" ,
    'C':"#C4EFFF" ,
    'Q':"#C4EFFF" ,
    'E':"#C4EFFF" ,
    'G':"#C4EFFF" ,
    'H':"#C4EFFF" ,
    'I':"#C4EFFF" ,
    'L':"#C4EFFF" ,
    'K':"#C4EFFF" ,
    'M':"#C4EFFF" ,
    'F':"#000000" ,
    'P':"#C4EFFF" ,
    'S':"#C4EFFF" ,
    'T':"#C4EFFF" ,
    'W':"#C4EFFF" ,
    'Y':"#C4EFFF" ,
    'V':"#C4EFFF" ,
    'B':"#C4EFFF" ,
    'Z':"#C4EFFF" ,
    'X':"#C4EFFF",
    '.':"#C4EFFF",
    '-':"#C4EFFF",
    }


# Code from ete package, mofifying to use the colors we set
class MySequenceFace(faces.StaticItemFace):
    """ Creates a new molecular sequence face object.


    :argument seq:  Sequence string to be drawn
    :argument seqtype: Type of sequence: "nt" or "aa"
    :argument fsize:   Font size,  (default=10)

    You can set custom colors for amino-acids or nucleotides:

    :argument None  codon       : a string that corresponds to the reverse translation of the amino-acid sequence
    :argument 11    col_w       : width of the column (if col_w is lower than font size, letter wont be displayed)
    :argument None  fg_colors   : dictionary of colors for foreground, with as keys each possible character in sequences, and as value the colors
    :argument None  bg_colors   : dictionary of colors for background, with as keys each possible character in sequences, and as value the colors
    :argument 3     alt_col_w   : works together with special_col option, defines the width of given columns
    :argument None  special_col : list of lists containing the bounds of columns to be displayed with alt_col_w as width
    :argument False interactive : more info can be displayed when mouse over sequence

    """
    def __init__(self, seq, seqtype="aa", fsize=10,
                 fg_colors=None, bg_colors=None,
                 codon=None, col_w=11, alt_col_w=3,
                 special_col=None, interactive=False):
        self.seq         = seq
        self.codon       = codon
        self.fsize       = fsize
        self.style       = seqtype
        self.col_w       = float(col_w)
        self.alt_col_w   = float(alt_col_w)
        self.special_col = special_col if special_col else []
        self.width       = 0 # will store the width of the whole sequence
        self.interact    = interactive

        if self.style == "aa":
            if not fg_colors:
                fg_colors = _aafgcolors
            if not bg_colors:
                bg_colors = _aabgcolors
        else:
            if not fg_colors:
                fg_colors = _ntfgcolors
            if not bg_colors:
                bg_colors = _ntbgcolors

        self.fg_col = self.__init_col(fg_colors)
        self.bg_col = self.__init_col(bg_colors)

        # for future?
        self.row_h       = 13.0

        super(MySequenceFace,
              self).__init__(QGraphicsRectItem(0, 0, self.width, self.row_h))

    def __init_col(self, color_dic):
        """to speed up the drawing of colored rectangles and characters"""
        new_color_dic = {}
        for car in color_dic:
            new_color_dic[car] = QBrush(QColor(color_dic[car]))
        return new_color_dic

    def update_items(self):
        #self.item = QGraphicsRectItem(0,0,self._total_w, self.row_h)
        seq_width = 0
        nopen = QPen(QtCore.Qt.NoPen)
        font = QFont("Courier", self.fsize)
        rect_cls = QGraphicsRectItem
        for i, letter in enumerate(self.seq):
            width = self.col_w
            for m in self.special_col:
                if m[0] < i <= m[1]:
                    width = self.alt_col_w
                    break
            #load interactive item if called correspondingly
            rectItem = rect_cls(0, 0, width, self.row_h, parent=self.item)
            rectItem.setX(seq_width) # to give correct X to children item
            rectItem.setBrush(self.bg_col[letter])
            rectItem.setPen(nopen)
            if self.interact:
                if self.codon:
                    rectItem.codon = '%s, %d: %s' % (self.seq[i], i,
                                                     self.codon[i*3:i*3+3])
                else:
                    rectItem.codon = '%s, %d' % (self.seq[i], i)
            # write letter if enough space
            if width >= self.fsize:
                text = QGraphicsSimpleTextItem(letter, parent=rectItem)
                text.setFont(font)
                text.setBrush(self.fg_col[letter])
                # Center text according to rectItem size
                tw = text.boundingRect().width()
                th = text.boundingRect().height()
                text.setPos((width - tw)/2, (self.row_h - th)/2)
            seq_width += width
        self.width = seq_width

In [26]:
# Get a subset of the alignment
def subset_alignment(alignment_file, start, end):
    genes = []
    seqs = []
    seq = ""

    # Read in alignment sequences and save to list
    with open(alignment_file, "r") as f:
        lines = f.readlines()
    for line in lines:
        if line.startswith(">"):
            genes.append(line.strip())
            if len(seq) > 0:
                seqs.append(seq)
                seq = ""
        else:
            seq += line.strip()
    seqs.append(seq)
    seq = ""

    # Write it to a string in fasta format
    fasta_txt = ""
    for gene, seq in zip(genes, seqs):
        fasta_txt += gene + "\n"
        fasta_txt += str(seq[start:end]) + "\n"
    
    return fasta_txt


In [41]:
moved_motif = ["Saccharomycotina_sp|P03069|GCN4_YEAST", 
                       "Saccharomycotina_jgi|Nadfu1|51114|gm1.2755_g",
                       "Saccharomycotina_jgi|Nadfu1|65686|estExt_Genemark1.C_4_t10334",
                       "Anig_EHA26660.1_AspergillusNigerATCC1015",
                       "Tdel_TDEL0H02940"]

def test_layout_phylo_aa(node):
    '''
    layout for CodemlTree
    '''
    if hasattr(node, "collapsed"):
        if node.collapsed == 1:
            node.img_style["draw_descendants"]= False
    if node.is_leaf:
        if hasattr (node, "sequence"):
            print(node.name)
            if node.name == "Saccharomycotina_sp|P03069|GCN4_YEAST":
                seqface =  MySequenceFace(node.sequence, "aa",
                                        fsize=10,
                                        col_w=11, interactive=False,
                                        bg_colors= _aabgcolors_highlight_sc)
            elif node.name in moved_motif:
                seqface =  MySequenceFace(node.sequence, "aa",
                                        fsize=10,
                                        col_w=11, interactive=False,
                                        bg_colors=_aabgcolors_highlight)
            else:
                seqface =  MySequenceFace(node.sequence, "aa",
                                      fsize=10,
                                      col_w=11, interactive=False)
            faces.add_face_to_node(seqface, node, 1, aligned=True)


alignment_file = "../data/fasta_files/AllSeqs_IntegralAround_WxxLF_-50_+20_top138_mafft.fasta"


# Define your start and end positions
start_position = 0
end_position = 138

fasta_txt = subset_alignment(alignment_file, start_position, end_position).replace("#", "_")

tree = PhyloTree("../data/phylogenetic_info/gcn4_gene_phylogeny.treefile", format=1)
tree.link_to_alignment(alignment=fasta_txt, alg_format="fasta")


# Create the TextFace objects for the start and end positions
start_annotation = faces.TextFace(f"{start_position}", fsize=10, fgcolor="black")
end_annotation = faces.TextFace(f"{end_position}", fsize=10, fgcolor="black")

# Pruning twice - branch lengths are not conserved
clade_leaves = [leaf for leaf in tree.get_leaf_names() if leaf in fasta_txt]
tree.prune(clade_leaves) # Here there are 138 species

clade_leaves = tree.get_common_ancestor(moved_motif).get_leaf_names() 
tree.prune(clade_leaves)

first_leaf = tree.get_leaves()[0]

ts = TreeStyle()
ts.layout_fn = test_layout_phylo_aa
# Add the start and end annotations to the first leaf
first_leaf.add_face(start_annotation, column=0, position="aligned")

# tree.show(tree_style=ts)
tree.render("../figures/FF_motif_alignment_tree_subset.pdf", tree_style=ts)

Warnning: [1050] terminal nodes could not be found in the alignment.


Dothideomycetes_jgi|Ternu1|295555|estExt_Genewise1.C_420077
Tmar_XP_002151511.1_TalaromycesMarneffeiATCC18224
Orbiliomycetes_tr|S8AP85|S8AP85_DACHA
Pezizomycetes_jgi|Ascim1|361388|estExt_Genemark1.C_290083
Eurotiomycetes_jgi|Theau1|5329|gm1.5329_g
Eurotiomycetes_jgi|Gymci1_1|302926|estExt_Genewise1.C_400046
Cimm_XP_012214147.1_CoccidioidesImmitisRS
Blastocladiomycota_jgi|Catan2|1506241|gm1.11555_g
Saccharomycotina_jgi|Lipst1_1|2531|gm1.2531_g
Saccharomycotina_jgi|Yarlip1|160666|gm1.4739_g
Eurotiomycetes_jgi|Penbr2|60600|gm1.7763_g
Kpas_emb_CCA37254.1_KomagataellaPastorisCBS7435
Saccharomycotina_jgi|Wicdo1|218662|gm1.821_g
Saccharomycotina_jgi|Canca1|2660|gm1.2660_g
Eurotiomycetes_jgi|Monpu1|465512|estExt_Genewise1.C_1320051
Eurotiomycetes_jgi|Penatra1|32388|gw1.2.1490.1
Saccharomycotina_jgi|Wican1|37412|estExt_Genewise1.C_1_t10265
Saccharomycotina_jgi|Yarlip1|99120|estExt_Genewise1.C_4_t10451
Saccharomycotina_jgi|Canca1|23981|e_gw1.2.2068.1
Saccharomycotina_jgi|Canca1|57326|fgenesh1_pm

{'nodes': [[0.5, 772.1666984558105, 4.5, 776.1666984558105, 0, None],
  [11.969321836163264,
   537.2621078491211,
   15.969321836163264,
   541.2621078491211,
   1,
   None],
  [24.173628179270814,
   341.0242156982422,
   28.173628179270814,
   345.0242156982422,
   2,
   None],
  [43.696868827863454, 12.0, 47.696868827863454, 16.0, 3, None],
  [98.00613363277569, 5.5, 102.00613363277569, 9.5, 4, None],
  [46.69691258351372, 18.5, 50.69691258351372, 22.5, 5, None],
  [41.24503555158139,
   670.0484313964844,
   45.24503555158139,
   674.0484313964844,
   6,
   None],
  [52.437552490388875,
   626.0968627929688,
   56.437552490388875,
   630.0968627929688,
   7,
   None],
  [73.7201791461299,
   572.3187255859375,
   77.7201791461299,
   576.3187255859375,
   8,
   None],
  [85.53211870856114,
   514.324951171875,
   89.53211870856114,
   518.324951171875,
   9,
   None],
  [91.81727903496714,
   451.14990234375,
   95.81727903496714,
   455.14990234375,
   10,
   None],
  [101.836380

In [ ]:
""